In [94]:
import pandas as pd

dataset = pd.read_csv('./n_movies.csv')

def format_stars(stars):
    stars = stars.split(', \'')
    stars = [star for star in stars if star != '' and star != '    Stars:\'']
    stars[0] = stars[0][2:]
    stars[-1] = stars[-1][:-2]

    dir_index = -1
    for j in range(len(stars)):
        stars[j] = stars[j].replace(', "', '')
        if 'Stars:\'' in stars[j]:
            stars[j] = stars[j].split(':\'')[1]
        
        if '|' in stars[j]:
            dir_index = j
    
    stars = stars[dir_index + 1:]

    return stars


unique_actors = []
def add_unique_actors(actors):
    for actor in actors:
        if actor not in unique_actors:
            unique_actors.append(actor)

dataset['actors'] = dataset['stars'].apply(format_stars)
dataset['actors'].apply(add_unique_actors)

print(unique_actors)

['Ralph Macchio', 'William Zabka', 'Courtney Henggeler', 'Xolo Maridueña', 'Claire Foy', 'Olivia Colman', 'Imelda Staunton', 'Matt Smith', 'Bob Odenkirk', 'Rhea Seehorn', 'Jonathan Banks', 'Patrick Fabian', 'Emily Deschanel', 'Sam Jaeger', 'Gerardo Celasco', 'Madeleine Arthur', 'Zach Aguilar', 'Kenichiro Ohashi', 'Emi Lo', 'Aoi Yûki', 'Tom Sturridge', 'Boyd Holbrook', 'Patton Oswalt', 'Vivienne Acheampong', 'Justin Roiland', 'Chris Parnell', 'Spencer Grammer', 'Sarah Chalke', 'Bryan Cranston', 'Aaron Paul', 'Anna Gunn', 'Betsy Brandt', 'Morgan Taylor Campbell', 'Italia Ricci', 'Rhianna Jagpal', 'Iñaki Godoy', 'Ana de Armas', 'Lucy DeVito', 'Garret Dillahunt', 'Adrien Brody', 'Millie Bobby Brown', 'Finn Wolfhard', 'Winona Ryder', 'Caleb McLaughlin', 'Queen Latifah', 'Ludacris', 'Mychala Lee', 'Shaun Dixon', 'Andrew Lincoln', 'Norman Reedus', 'Melissa McBride', 'Lauren Cohan', 'Daniel Craig', 'Edward Norton', 'Kate Hudson', 'Dave Bautista', 'Camila Mendes', 'Maya Hawke', 'Austin Abrams',